In [7]:
import os
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [8]:
OUTPUT_PATH = os.path.join(os.getcwd(), "data", "inputs")
PATHS = [pathname for pathname in os.listdir(OUTPUT_PATH) if os.path.isdir(os.path.join(OUTPUT_PATH, pathname))]
DATASET_TYPES = ["betadgp_covdgp_data", "betadgp_beta2x2_data", "betadgp_data"]
y_name = "betas_dgp"

In [9]:
p = PATHS[0]
d = DATASET_TYPES[0]

tmp_data = pd.read_csv(os.path.join(OUTPUT_PATH, p, d + ".csv"))
tmp_data.head()

,Var1,Var2,betas_dgp,cov_dgp
0,1,1,0.0,1.000000
1,1,2,0.0,-0.082149
2,1,3,0.0,-0.002405
3,1,4,0.0,-0.013750
4,1,5,0.0,0.046244


In [10]:
y = tmp_data[y_name]
X = tmp_data.drop([y_name, "Var1", "Var2"], axis=1)

## 1. Baseline neural network model

In [11]:
def baseline_model():
    model = Sequential()
    model.add(Dense(5,
                    input_dim=X.shape[1],
                    kernel_initializer='normal',
                    activation='relu'))
    model.add(Dense(1,
                    kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

# cross-validation
estimator = KerasRegressor(build_fn=baseline_model,
                           epochs=100,
                           batch_size=100,
                           verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator,
                          X,
                          y,
                          cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -0.00 (0.00) MSE
